<a href="https://colab.research.google.com/github/ganbozza/test/blob/main/Mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# !sudo apt-get update && sudo apt-get install megacmd
#!wget https://mega.nz/linux/repo/Arch_Extra/x86_64/megacmd-x86_64.pkg.tar.zst && sudo pacman -U "$PWD/megacmd-x86_64.pkg.tar.zst"
#!wget https://mega.nz/linux/repo/xUbuntu_22.04/amd64/megacmd-xUbuntu_22.04_amd64.deb && sudo apt install "$PWD/megacmd-xUbuntu_22.04_amd64.deb"
#!sudo -v ; curl https://rclone.org/install.sh | sudo bash

import os

if os.path.exists("/usr/bin/rclone"):
  if not os.path.exists("/root/.config/rclone/rclone.conf"):
    if not os.path.exists("./rclone.conf"):
      print("Original config not found")
    else:
      print("Copying config to /root/.config/rclone")
      !cp ./rclone.conf /root/.config/rclone/rclone.conf

  else:
    print("Using existing config")
else:
  print("Installing rclone")

if not os.path.exists("/content/mega-drive"):
  !mkdir /content/mega-drive

!nohup rclone --vfs-cache-mode writes mount mega: ./mega-drive &
#!ls /content/mega-drive/

Using existing config
nohup: appending output to 'nohup.out'


In [19]:
!ls ./mega-drive    #after sucessfully mounted onedrive then show your onedrive folders and files

test
